In [ ]:
from selenium import webdriver as wd
import cx_Oracle
import os
import time
import re
p = re.compile('만족')
p1 = re.compile("2019|2018")
p2 = re.compile("결제완료")
p3 = re.compile("일")
p4 = re.compile("건")
p5 = re.compile("조회수")
p6 = re.compile("장바구니")
os.putenv("NLS_LANG",".UTF8")
connection = cx_Oracle.connect('hr' , '1234' , 'localhost/xe')
cursor = connection.cursor()
#접속할 주소
url = "https://store.musinsa.com/app/items/lists/001005"
#자동으로 웹 동작을 할 수 있다
driver = wd.Chrome(executable_path="chromedriver.exe")
#사이트 로드를 10초동안 기다리고 10초 안에 로드 완료시 다음 라인 실행
driver.implicitly_wait(10)
#url에 접속 하겠따.
driver.get(url)
cloth_container = driver.find_element_by_class_name("list-box")
cloth_list = cloth_container.find_elements_by_class_name("li_box")
total_page = int(driver.find_element_by_class_name("is_goods").text.split(":")[-1].replace("개","").replace(",",""))
if(total_page % 80 == 0):
    total_page = total_page // 80
else:
    total_page = (total_page // 80) +1
for page in range(1,(total_page+1)):
    driver.execute_script("listSwitchPage(document.f1, '{}')".format(page))
    time.sleep(3)
    for i in range(0,80):
        try:
            cloth_container = driver.find_element_by_class_name("list-box")
            cloth_list = cloth_container.find_elements_by_class_name("li_box")
            cloth_list[i].find_element_by_class_name("img-block").click()
            print("brand: ",driver.find_element_by_css_selector(".product_article>li>.product_article_contents>strong>a"))

        except:
            time.sleep(3)
            driver.refresh()
            cloth_container = driver.find_element_by_class_name("list-box")
            cloth_list = cloth_container.find_elements_by_class_name("li_box")
            cloth_list[i].find_element_by_class_name("img-block").click()
            print("brand: ",driver.find_element_by_css_selector(".product_article>li>.product_article_contents>strong>a"))



        brand = driver.find_element_by_css_selector(".product_article>li>.product_article_contents>strong>a").text
        print("season: ",driver.find_element_by_class_name("product_article"))
        season_div = driver.find_element_by_class_name("product_info_section")
        season_Ul = season_div.find_element_by_class_name("product_article")
        season_li_list = season_Ul.find_elements_by_tag_name("li")
        season = ""
        total_sell = ""
        review = ""
        bucket = ""
        popularity = ""
        print(len(season_li_list))
        for index in range(0,len(season_li_list)):
            result = season_li_list[index].find_element_by_class_name("product_article_contents")
            season_test = result.text
            if p1.search(season_test) != None:
                if(season_test.split("/")[-1].replace(" ","") == "남"):
                    season = season_test.replace("남","").replace(" ","")
            if p2.search(season_test) != None:
                total_sell = int(season_test.split("개")[0].replace(",",""))
            if p5.search(season_test) != None:
                popularity = int(season_test.split("(조회수")[0].replace(",",""))
                bucket = int(season_test.split("(장바구니")[0].split("/")[1].replace(" ","").replace(",",""))

        sex_container = driver.find_element_by_class_name("txt_gender")
        sex = sex_container.find_element_by_tag_name("span").text


        title = driver.find_element_by_class_name("product_title").text

        like_ = ""
        try:
            like_ = int(driver.find_element_by_class_name("prd_like_cnt").text.replace(",",""))
        except:
            like_ = ""
        satisfaction = ""
        satisfaction_list = driver.find_elements_by_class_name("product_article_contents")
        for index in satisfaction_list:
            if p.search(index.text) != None:
                satisfaction = float(index.text.split("/")[1].split("%")[0].replace(" ",""))
                review = int(index.text.split("/")[0].split("건")[0].replace(",",""))
                break;

        img_url_div = driver.find_element_by_class_name("product-img")
        img_url = img_url_div.find_element_by_tag_name("img").get_attribute("src")

        delivery_date_container = driver.find_element_by_class_name("delivery_info_section")
        dLi_list = delivery_date_container.find_elements_by_tag_name("li")
        delivery_date = ""
        for index in range(0,len(dLi_list)):
            result = dLi_list[index].find_element_by_class_name("product_article_contents")
            test = result.text
            if(p3.search(test)) != None:
                delivery_date = float(test.split("일")[0])
        price_container = driver.find_element_by_class_name("price_info_section")
        pLi_list = price_container.find_elements_by_tag_name("li")
        price = ""
        sale = ""
        print(len(dLi_list))
        print(len(pLi_list))
        for li in pLi_list:
            print(li.text)
        try:
            if(len(pLi_list) == 6 or len(pLi_list) == 7):
                price = int(driver.find_element_by_id("sale_price").text.split("원")[0].replace(",",""))
                sale = float((driver.find_element_by_class_name("txt_kor_discount").text.split("%")[0]))

            else:
                price = driver.find_element_by_id("goods_price").text.replace("원","").replace(",","")
                sale = ""
        except:
            price = int(driver.find_element_by_id("goods_price").text.replace("원","").replace(",",""))
            sale = ""



        category_container = driver.find_element_by_class_name('item_categories')
        category_ = category_container.find_elements_by_tag_name("a")[1].text


        print("num: " , i)
        print("brand: " , brand)
        print("season: " , season)
        print("sex: " , sex)
        print("title: " , title)
        print("popularity: " , popularity)
        print("bucket: " , bucket)
        print("like_: " , like_)
        print("review: " , review)
        print("satisfaction: " , satisfaction)
        print("total_sell: " , total_sell)
        print("delivery_date: " , delivery_date)
        print("price",price)
        print("sale" , sale)
        print("category_",category_)
        print("img_url",img_url)
        print("="*130)
        
        driver.back()
        time.sleep(3)

brand:  <selenium.webdriver.remote.webelement.WebElement (session="77b5bbbf5cac97b4712d75b8cfd33e42", element="0.33548824770102326-1")>
season:  <selenium.webdriver.remote.webelement.WebElement (session="77b5bbbf5cac97b4712d75b8cfd33e42", element="0.33548824770102326-2")>
4
2
5

-무신사 판매가 42,900 원
-무신사 회원가 39,039 ~ 42,042 원 ▼
ㄴ 바로접속 ON 시 무신사 회원가가 적용됩니다.
ㄴ 쿠폰 사용이 허용된 상품입니다. [보유 쿠폰을 확인] 하세요.

-무신사 적립금 최대 1,716원 (바로접속 ON, 로그인 필요)
num:  0
brand:  UNI APPAREL
season:  2019F/W/
sex:  남
title:  베이직 기모 터틀넥 스웨트셔츠 카키|BASIC TUMBLE 950G TURTLENECK SWEAT SHIRTS KHAKI
popularity:  
bucket:  
like_:  
review:  
satisfaction:  
total_sell:  
delivery_date:  1.7
price 42900
sale 
category_ 맨투맨/스웨트셔츠
img_url https://image.msscdn.net/images/goods_img/20191211/1249022/1249022_1_500.jpg
brand:  <selenium.webdriver.remote.webelement.WebElement (session="77b5bbbf5cac97b4712d75b8cfd33e42", element="0.6993723837552825-1")>
season:  <selenium.webdriver.remote.webelement.WebElement (session="77b5bbbf5cac97b4712d7

brand:  <selenium.webdriver.remote.webelement.WebElement (session="77b5bbbf5cac97b4712d75b8cfd33e42", element="0.6995119427502907-1")>
season:  <selenium.webdriver.remote.webelement.WebElement (session="77b5bbbf5cac97b4712d75b8cfd33e42", element="0.6995119427502907-2")>
4
2
5

-무신사 판매가 148,000 원
-무신사 회원가 134,680 ~ 145,040 원 ▼
ㄴ 바로접속 ON 시 무신사 회원가가 적용됩니다.
ㄴ 쿠폰 사용이 허용된 상품입니다. [보유 쿠폰을 확인] 하세요.

-무신사 적립금 최대 5,920원 (바로접속 ON, 로그인 필요)
num:  9
brand:  STIGMA
season:  2019F/W/
sex:  남
title:  CREAM OVERSIZED WOOL TURTLE NECK SWEATER IVORY|CREAM OVERSIZED WOOL TURTLE NECK SWEATER IVORY
popularity:  
bucket:  
like_:  
review:  
satisfaction:  
total_sell:  
delivery_date:  1.5
price 148000
sale 
category_ 맨투맨/스웨트셔츠
img_url https://image.msscdn.net/images/goods_img/20191211/1248914/1248914_1_500.jpg
brand:  <selenium.webdriver.remote.webelement.WebElement (session="77b5bbbf5cac97b4712d75b8cfd33e42", element="0.19639871638264483-1")>
season:  <selenium.webdriver.remote.webelement.WebElement (sessio